In [15]:
from math import sqrt

#from bertviz.transformers_neuron_view import BertModel
#from bertviz.neuron_view import show
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoConfig, AutoTokenizer

In [2]:
model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
#model = BertModel.from_pretrained(model_ckpt)

In [3]:
text = 'time flies like an arrow'
#show(
#    model, 
#    'bert',
#    tokenizer,
#    text,
#    display_mode='light',
#    layer=0,
#    head=8)

In [5]:
inputs = tokenizer(text, return_tensors='pt', add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [7]:
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [8]:
input_embeds = token_emb(inputs.input_ids)
input_embeds.size() # one batch of our 5 words in a 768-vector encoding

torch.Size([1, 5, 768])

In [13]:
query = key = value = input_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

In [14]:
scores

tensor([[[28.3149, -0.2552, -1.0394, -0.1332,  1.4793],
         [-0.2552, 28.3434, -0.4528, -0.1831,  0.1712],
         [-1.0394, -0.4528, 28.4870, -0.2822, -0.3228],
         [-0.1332, -0.1831, -0.2822, 30.7570, -0.4302],
         [ 1.4793,  0.1712, -0.3228, -0.4302, 29.4271]]],
       grad_fn=<DivBackward0>)

In [16]:
weights = F.softmax(scores, dim=-1)
weights

tensor([[[1.0000e+00, 3.9100e-13, 1.7849e-13, 4.4174e-13, 2.2155e-12],
         [3.7999e-13, 1.0000e+00, 3.1185e-13, 4.0842e-13, 5.8205e-13],
         [1.5026e-13, 2.7013e-13, 1.0000e+00, 3.2040e-13, 3.0765e-13],
         [3.8419e-14, 3.6550e-14, 3.3101e-14, 1.0000e+00, 2.8548e-14],
         [7.2851e-13, 1.9694e-13, 1.2017e-13, 1.0794e-13, 1.0000e+00]]],
       grad_fn=<SoftmaxBackward0>)

In [17]:
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [18]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [19]:
# wrap it up
def scaled_dot_product_attention(query, key, value):
    dim_k = key.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    attn = torch.bmm(weights, value)
    return attn